<a href="https://colab.research.google.com/github/TirendazAcademy/Tracking-Agents-with-Comet/blob/main/tracking_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U langchain-openai openai langchain tiktoken comet_llm google-search-results cohere chromadb tensorflow-probability kaleido python-multipart

# Importing the necessary libraries

In [2]:
import os
import comet_llm

os.environ["COMET_API_KEY"] = "..."
os.environ["OPENAI_API_KEY"] = "..."
os.environ["SERPAPI_API_KEY"] = "..."

comet_llm.init(project="comet-langchain-agents")

# Scraping the data

In [3]:
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page to scrape
url = 'https://en.wikipedia.org/wiki/MrBeast'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the text on the page
text = soup.get_text()

# Open a new file called 'output.txt' in write mode and store the file object in a variable
with open('output.txt', 'w', encoding='utf-8') as file:
    # Write the string to the file
    file.write(text)

# load the document
with open('./output.txt', encoding='utf-8') as f:
    text = f.read()

# Data ingestion

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# define the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.create_documents([text])

# define the embeddings model
embeddings = OpenAIEmbeddings()

# use the text chunks and the embeddings model to fill our vector store
db = Chroma.from_documents(texts, embeddings)

# Tracking an agent with Comet ML

In [7]:
from langchain_openai import OpenAI
from langchain.agents import initialize_agent, load_tools
from langchain import PromptTemplate
from langchain.callbacks import CometCallbackHandler, StdOutCallbackHandler
#from langchain.callbacks.tracers.comet import CometTracer

comet_callback = CometCallbackHandler(
    project_name="comet-langchain-agents",
    complexity_metrics=True,
    stream_logs=True,
    tags=["agent"],
)

# Create a comet callback
callbacks = [StdOutCallbackHandler(), comet_callback]

# Define an openai llm
llm = OpenAI(temperature=0.9, callbacks=callbacks)

# Define a question
users_question = "What is MrBeast's age to the power of 0.54?"

# use our vector store to find similar text chunks
results = db.similarity_search(
    query=users_question,
)

# define the prompt template
template = """

You are a chatbot who loves helping people! Given the context sections below, \
answer the question using only the context provided.

Context sections:
{context}

Question:
{users_question}

Answer:
"""

# Create a prompt template
prompt = PromptTemplate(template=template, input_variables=["context", "users_question"])

# Fill the prompt template
prompt_text = prompt.format(context = results, users_question = users_question)

# Load tools
tools = load_tools(["serpapi", "llm-math"], llm=llm, callbacks=callbacks, verbose=True)

# Initialize an agent
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    callbacks=callbacks,
    verbose=True,
)

# Run the agent
agent.run(prompt_text, callbacks=callbacks,)
comet_callback.flush_tracker(agent, finish=True)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/tirendaz-test/comet-langchain-agents/e61921622cdf4eea8fe6e838b19d9276

COMET WARNING: The comet_ml callback is currently in beta and is subject to change based on updates to `langchain`. Please report any issues to https://github.com/comet-ml/issue-tracking/issues with the tag `langchain`.
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 

You are a chatbot who loves helping people! Given the context sections below, answer the question using only the context provided.

Context sections:
[Document(pag


> Finished chain.
 I need to find out MrBeast's age to the power of 0.54.
Action: Search
Action Input: "MrBeast age"

COMET WARNING: Unexpected data format provided! Input Value for stop will not be logged



Observation: 25 years
Thought:

> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 

You are a chatbot who loves helping people! Given the context sections below, answer the question using only the context provided.

Context sections:
[Document(page_content

...


> Finished chain.
 I need to calculate 25 to the power of 0.54
Action: Calculator
Action Input: 25^0.54

> Entering new LLMMathChain chain...
25^0.54

> Entering new LLMChain chain...
Prompt after formatting:
Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${Question with math problem.}
```text
${single line mathematical expression that solves the problem}
```
...numexpr.evaluate(text)...
```output
${Output of running the code}
```
Answer: ${Answer}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: 25^0.54



COMET WARNING: Unexpected data format provided! Input Value for stop will not be logged



> Finished chain.
```text
25**0.54
```
...numexpr.evaluate("25**0.54")...

Answer: 5.687057308780144
> Finished chain.

Observation: Answer: 5.687057308780144
Thought:

> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 

You are a chatbot who loves helping


> Finished chain.
 I now know the final answer
Final Answer: 5.687057308780144

> Finished chain.


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/tirendaz-test/comet-langchain-agents/e61921622cdf4eea8fe6e838b19d9276
COMET INFO:   Others:
COMET INFO:     Created from    : langchain
COMET INFO:     hasNestedParams : True
COMET INFO:   Parameters:
COMET INFO:     model__type             : openai
COMET INFO:     model_frequency_penalty : 0
COMET INFO:     model_logit_bias        : {}
COMET INFO:     model_max_tokens        : 256
COMET INFO:     model_model_name        : gpt-3.5-turbo-instruct
COMET INFO:     model_n                 : 1
COMET INFO:     model_presence_penalty  : 0
COMET INFO:     model_temperature       : 0.9
COMET INFO:     model_top_p             : 1
COMET IN

## Resources

- [All You Need to Know to Build Your First LLM App](https://towardsdatascience.com/all-you-need-to-know-to-build-your-first-llm-app-eb982c78ffac)
- [Conversation QA Gradio ](https://github.com/hwchase17/conversation-qa-gradio/tree/master)

Thanks for reading. Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) 😎